In [1]:
# feature selection

# 필요한 라이브러리를 임포트합니다.
import pandas as pd
import numpy as np
from scipy.stats import bartlett
import scipy.stats as stats
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import warnings

# 경고 메시지를 무시하기 위해 설정합니다.
warnings.simplefilter('ignore')

# train_data.csv 파일을 읽어옵니다. 인덱스 컬럼은 첫 번째 컬럼입니다.
df = pd.read_csv("train_data.csv", index_col=0)

# 데이터 프레임의 정보를 출력합니다. (데이터 타입, 결측치 여부 등을 확인할 수 있습니다.)
df.info()

# 바틀렛 검정을 수행하는 함수입니다.
# col: 검정할 컬럼 리스트, p_value: 유의 수준, H: 귀무가설(0: 등분산, 1: 이분산)
def bartlett_test(col, p_value=0.05, H=1):
    result_list = []
    for i in col:
        # 바틀렛 검정을 수행합니다.
        T, p_val = bartlett(df[df['label'] == 1][i], df[df['label'] == 0][i]) 
        result_list.append([i, p_val])

    result_df = pd.DataFrame(result_list, columns=['변수', 'p_value'])
    # H가 1이면 이분산성 변수, 0이면 등분산성 변수를 필터링합니다.
    filtered_df = result_df[result_df['p_value'] < p_value] if H else result_df[result_df['p_value'] >= p_value]
    return filtered_df.sort_values('p_value')

# 이분산성 변수와 등분산성 변수를 구분합니다.
x_hetero = bartlett_test(df.columns, H=1)
x_homo = bartlett_test(df.columns, H=0)

# t-검정을 수행하는 함수입니다.
# col: 검정할 컬럼 리스트, col_h0: 등분산성 변수, col_h1: 이분산성 변수, p_value: 유의 수준
def t_test(col, col_h0, col_h1, p_value=0.05):
    result_list = []
    for i in col:
        # 해당 변수가 등분산성 변수에 속하면 wald t-test를, 이분산성 변수에 속하면 welch’s t-test를 수행합니다.
        equal_var = (col_h0['변수'] == i).any()
        t_stat, p_val = stats.ttest_ind(df[df['label'] == 1][i], df[df['label'] == 0][i], equal_var=equal_var)
        result_list.append([i, p_val])

    result_df = pd.DataFrame(result_list, columns=['변수', 'p_value'])
    filtered_df = result_df[result_df['p_value'] < p_value]
    return filtered_df.sort_values('p_value')

# 유의미한 변수를 선택합니다. p_value가 0.1보다 작은 변수들을 추출합니다.
x_ttest = t_test(df.columns, x_homo, x_hetero, p_value=0.1)
print("유의한 피처 수 :", len(x_ttest))
x_ttest.sort_values(by="변수", ascending=True)

# 로지스틱 회귀 모델을 설정하고, 그리드 서치로 최적의 매개변수를 찾습니다.
# 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(df.drop('label', axis=1), df['label'], test_size=0.3, random_state=42)

# 로지스틱 회귀 모델 설정 및 그리드 서치
lasso_model = LogisticRegression()
param_grid = {'penalty': ['l1'], 'C': [0.001, 0.01, 0.1, 1, 2, 5, 10], 'solver': ['liblinear']}
grid_search = GridSearchCV(lasso_model, param_grid=param_grid, return_train_score=True, cv=5)
grid_search.fit(X_train, y_train)

# 그리드 서치 결과를 정리합니다.
grid_search_result = pd.DataFrame(grid_search.cv_results_)
grid_search_result = grid_search_result.sort_values(by='rank_test_score')
grid_search_result[['params', 'mean_train_score', 'mean_test_score', 'rank_test_score']]

# 최적의 파라미터와 정확도를 출력합니다.
print('GridSearchCV 최적 파라미터:', grid_search.best_params_)
print('GridSearchCV 최고 정확도:{0:.4f}'.format(grid_search.best_score_))

# 라쏘 모델로 선택된 피처를 추출합니다.
lasso_best = LogisticRegression(C=0.001, penalty='l1', solver='liblinear').fit(X_train, y_train)
df_lasso = pd.DataFrame({'feature': X_train.columns, 'coef': lasso_best.coef_[0]})
df_lasso = df_lasso[df_lasso['coef'] != 0]

# 라쏘에서 선택된 피처의 수를 출력합니다.
print('Lasso에서 선택된 피처 수:', len(df_lasso), '개')

# t-test와 라쏘에서 선택된 피처를 결합하여 최종 피처를 선택합니다.
def func_ttest(x):
    return 1 if x in x_ttest['변수'].values else 0

def func_lasso(x):
    return 1 if x in df_lasso['feature'].values else 0

feature_counts = pd.DataFrame({'Feature': X_train.columns})
feature_counts['ttest'] = feature_counts['Feature'].apply(func_ttest)
feature_counts['lasso'] = feature_counts['Feature'].apply(func_lasso)
feature_counts['total'] = feature_counts['ttest'] + feature_counts['lasso']

# 두 방법 모두에서 선택된 피처를 최종적으로 선택합니다.
feature_final = feature_counts[feature_counts['total'] >= 2]
print("선택된 피처 수 :", len(feature_final))
feature_final


<class 'pandas.core.frame.DataFrame'>
Index: 1566 entries, 2.4 to -2.98
Data columns (total 66 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   총자본순이익률_비율          1566 non-null   float64
 1   총자본순이익률_상위10개_차이    1566 non-null   float64
 2   총자본순이익률_상위10개_변동성   1566 non-null   float64
 3   총자본순이익률_산업평균_차이     1566 non-null   float64
 4   총자본순이익률_산업평균_변동성    1566 non-null   float64
 5   매출액증가율_차이           1566 non-null   float64
 6   매출액증가율_비율           1566 non-null   float64
 7   매출액증가율_상위10개_차이     1566 non-null   float64
 8   매출액증가율_상위10개_변동성    1566 non-null   float64
 9   매출액증가율_산업평균_차이      1566 non-null   float64
 10  매출액증가율_산업평균_변동성     1566 non-null   float64
 11  자기자본순이익률_차이         1566 non-null   float64
 12  자기자본순이익률_비율         1566 non-null   float64
 13  자기자본순이익률_상위10개_차이   1566 non-null   float64
 14  자기자본순이익률_상위10개_변동성  1566 non-null   float64
 15  자기자본순이익률_산업평균_차이    1566 non-null   float64
 16  자기자본순이익률

,Feature,ttest,lasso,total
15,자기자본순이익률_산업평균_차이,1,1,2
19,매출액순이익률_상위10개_차이,1,1,2
20,매출액순이익률_상위10개_변동성,1,1,2
60,순이익증가율_비율,1,1,2
64,순이익증가율_산업평균_변동성,1,1,2


In [2]:
# modeling

# 필요한 라이브러리들을 임포트합니다.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE

# 경고 메시지를 무시하기 위한 설정
import warnings
warnings.filterwarnings('ignore')

# 시각화를 위한 기본 설정
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

# 데이터 불러오기
train_df = pd.read_csv('train_data.csv', index_col=0)
test_df = pd.read_csv('test_수축기.csv', index_col=0)

# 데이터 불균형 확인
print(train_df['label'].value_counts())
train_df['label'].value_counts(normalize=True).plot(kind='bar')
print(train_df['label'].value_counts(normalize=True) * 100)

# 피처와 레이블 분리
X_train = train_df.drop(columns='label', axis=1)
y_train = train_df['label']
X_val = test_df.drop(columns='label', axis=1)
y_val = test_df['label']

# 선택한 피처만 사용
selected_features = ['자기자본순이익률_산업평균_차이', '매출액순이익률_상위10개_변동성', '순이익증가율_비율', '순이익증가율_산업평균_변동성']
X_train = X_train[selected_features]
X_val = X_val[selected_features]

# 피처 표준화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# SMOTE를 이용한 오버샘플링
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

# 모델별 하이퍼파라미터 그리드 설정
param_grid_svc = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': [0.1, 1, 'scale']}
param_grid_logreg = {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2'], 'solver': ['liblinear', 'saga']}

# 모델 목록
models = {'SVC': [SVC(), param_grid_svc], 'Logistic Regression': [LogisticRegression(), param_grid_logreg]}

# 결과를 저장할 리스트
results = []

# 각 모델별로 GridSearchCV 실행 및 평가
for name, (model, params) in models.items():
    clf = GridSearchCV(model, params, scoring='accuracy', cv=5)
    clf.fit(X_train_res, y_train_res)
    print(f"Best parameters for {name}: {clf.best_params_}")

    # 최적의 모델로 예측 및 성능 평가
    best_model = clone(model).set_params(**clf.best_params_)
    best_model.fit(X_train_res, y_train_res)
    predictions = best_model.predict(X_val_scaled)
    results.append([name, accuracy_score(y_val, predictions), precision_score(y_val, predictions), recall_score(y_val, predictions), f1_score(y_val, predictions)])
    train_df[name + '_pred'] = best_model.predict(X_train_scaled)

# 결과 출력
results_df = pd.DataFrame(results, columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])
print(results_df)

# 오버샘플링된 데이터 클래스 분포 확인
y_train_res_series = pd.Series(y_train_res)
print(y_train_res_series.value_counts())
y_train_res_series.value_counts(normalize=True).plot(kind='bar')
print(y_train_res_series.value_counts(normalize=True) * 100)


AttributeError: module 'pandas.core.strings' has no attribute 'StringMethods'